<a href="https://colab.research.google.com/github/thanxjiny/Vehicle-Damage-Detection/blob/main/notebooks/00_dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## 테스트

# [Cell 1] 설정 및 라이브러리 설치
# ---------------------------------------------------------
import os
import sys

# 1. 환경 감지 (Colab vs Local)
try:
    from google.colab import drive
    IS_COLAB = True
    print("✅ Google Colab 환경이 감지되었습니다.")

    # 드라이브 마운트 (데이터 저장을 위해 필수)
    drive.mount('/content/drive')

    # [사용자 수정 필요] 구글 드라이브 내 프로젝트 루트 경로
    # 예: 내 드라이브 > Project > Vehicle_Damage
    PROJECT_ROOT = "/content/drive/MyDrive/03. HDMF/(pre_study)2026_ImageDetectionStudy/1_car_classification"

except ImportError:
    IS_COLAB = False
    print("💻 로컬(PC) 환경입니다.")
    PROJECT_ROOT = "./data_workspace" # 로컬 작업시 데이터가 저장될 위치

# 2. 필수 라이브러리 설치 (실행 시 자동 설치)
!pip install fiftyone opencv-python tqdm

print(f"\n📂 작업 경로(PROJECT_ROOT): {PROJECT_ROOT}")
if not os.path.exists(PROJECT_ROOT):
    os.makedirs(PROJECT_ROOT)
    print("   └─ 경로가 없어서 새로 생성했습니다.")

✅ Google Colab 환경이 감지되었습니다.
Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
# [Cell 2] 데이터셋 구축 로직 (SyntaxError 수정 완료)
# ---------------------------------------------------------
import shutil
import cv2
import fiftyone as fo
import fiftyone.zoo as foz
from tqdm import tqdm
import random
import os

# =========================================================
# [설정] 파라미터
# =========================================================

# 1. AI-Hub 원본 데이터 경로
# 주의: 이 경로는 Colab에서 해당 폴더가 실제로 마운트되어 있어야 접근 가능합니다.
SRC_ROOT = "/content/drive/MyDrive/02. ML/3. Project/202511_ObjectDetection_Car_Sample/Car_Dataset"
SRC_IMG_DIR = os.path.join(SRC_ROOT, "images")
SRC_LBL_DIR = os.path.join(SRC_ROOT, "labels")

# 2. 최종 데이터가 저장될 경로
TARGET_ROOT = os.path.join(PROJECT_ROOT, "Dataset_No_Resizing")

COUNTS = {
    "damaged": 1200,   # AI-Hub 파손 차량 목표치
    "normal": 600,     # COCO 정상 차량 목표치
    "background": 600  # COCO 배경 이미지 목표치
}

# 차량 크기 필터링 (전체 면적의 5% 이상만 사용)
MIN_AREA_THRESHOLD = 0.05

# =========================================================
# [헬퍼 함수] 라벨 파일 찾기
# =========================================================
def find_matching_label(filename_no_ext, label_dir):
    """이미지 파일명과 매칭되는 라벨 파일 찾기"""
    if not os.path.exists(label_dir): return None

    for f in os.listdir(label_dir):
        if os.path.splitext(f)[0] == filename_no_ext:
            return f
    return None

# =========================================================
# [실행] 데이터셋 생성 함수
# =========================================================
def prepare_dataset():
    print(f"🚀 데이터셋 구축 시작")
    print(f"   소스(AI-Hub): {SRC_ROOT}")
    print(f"   타겟(Target): {TARGET_ROOT}")

    # 폴더 생성
    dirs = {
        "damaged": os.path.join(TARGET_ROOT, "damaged"),
        "normal": os.path.join(TARGET_ROOT, "normal"),
        "bg": os.path.join(TARGET_ROOT, "background")
    }

    for k, p in dirs.items():
        os.makedirs(os.path.join(p, "images"), exist_ok=True)
        if k == "damaged":
            os.makedirs(os.path.join(p, "labels"), exist_ok=True)

    # -----------------------------------------------------
    # 1. AI-Hub 파손 차량 처리
    # -----------------------------------------------------
    print("\n[1/3] 파손 차량 데이터 처리...")

    if os.path.exists(SRC_IMG_DIR) and os.path.exists(SRC_LBL_DIR):
        all_images = [f for f in os.listdir(SRC_IMG_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(all_images)

        processed_count = 0
        pbar = tqdm(all_images, desc="AI-Hub Copying")

        for img_file in pbar:
            if processed_count >= COUNTS["damaged"]: break

            filename_no_ext = os.path.splitext(img_file)[0]
            lbl_file = find_matching_label(filename_no_ext, SRC_LBL_DIR)

            if lbl_file is None: continue

            src_img_path = os.path.join(SRC_IMG_DIR, img_file)
            src_lbl_path = os.path.join(SRC_LBL_DIR, lbl_file)

            dst_img_path = os.path.join(dirs["damaged"], "images", img_file)
            dst_lbl_path = os.path.join(dirs["damaged"], "labels", lbl_file)

            shutil.copy(src_img_path, dst_img_path)
            shutil.copy(src_lbl_path, dst_lbl_path)

            processed_count += 1

        print(f"   ✅ 파손 차량 {processed_count}장 복사 완료.")
    else:
        print("   ⚠️ 지정된 경로에 AI-Hub 원본 데이터가 없습니다. (Skip)")
        print(f"      확인 경로: {SRC_IMG_DIR}")

    # -----------------------------------------------------
    # 2. COCO 정상 차량 다운로드
    # -----------------------------------------------------
    print("\n[2/3] COCO 정상 차량 다운로드 (FiftyOne)...")
    try:
        dataset = foz.load_zoo_dataset(
            "coco-2017", split="validation", classes=["car"],
            max_samples=COUNTS["normal"] * 3, shuffle=True
        )

        count = 0
        for sample in dataset:
            if count >= COUNTS["normal"]: break

            img = cv2.imread(sample.filepath)
            if img is None: continue

            # BBox 면적 계산
            max_bbox_area = 0
            if sample.ground_truth:
                for det in sample.ground_truth.detections:
                    if det.label == 'car':
                        area = det.bounding_box[2] * det.bounding_box[3]
                        if area > max_bbox_area: max_bbox_area = area

            if max_bbox_area >= MIN_AREA_THRESHOLD:
                shutil.copy(sample.filepath, os.path.join(dirs["normal"], "images", f"norm_{count}.jpg"))
                count += 1

        print(f"   ✅ 정상 차량 {count}장 수집 완료")

    except Exception as e:
        print(f"   ❌ 오류 발생: {e}")

    # -----------------------------------------------------
    # 3. COCO 배경 다운로드 (수정된 부분)
    # -----------------------------------------------------
    print("\n[3/3] 배경 이미지 다운로드...")
    try:
        bg_classes = ["bench", "traffic light", "cat", "dog"]
        # 기존 오류 부분 수정: split=\"validation\" -> split="validation"
        dataset_bg = foz.load_zoo_dataset(
            "coco-2017", split="validation", classes=bg_classes,
            max_samples=COUNTS["background"] + 100, shuffle=True
        )

        count = 0
        for sample in dataset_bg:
            if count >= COUNTS["background"]: break

            shutil.copy(sample.filepath, os.path.join(dirs["bg"], "images", f"bg_{count}.jpg"))
            count += 1

        print(f"   ✅ 배경 {count}장 수집 완료")

    except Exception as e:
        print(f"   ❌ 오류 발생: {e}")

    print(f"\n🎉 모든 작업 완료! 저장 위치: {TARGET_ROOT}")

# 실행
prepare_dataset()

/usr/local/lib/python3.12/dist-packages/glob2/fnmatch.py:141: SyntaxWarning: invalid escape sequence '\Z'
  return '(?ms)' + res + '\Z'


🚀 데이터셋 구축 시작
   소스(AI-Hub): /content/drive/MyDrive/02. ML/3. Project/202511_ObjectDetection_Car_Sample/Car_Dataset
   타겟(Target): /content/drive/MyDrive/03. HDMF/(pre_study)2026_ImageDetectionStudy/1_car_classification/Dataset_No_Resizing

[1/3] 파손 차량 데이터 처리...


AI-Hub Copying: 100%|██████████| 1200/1200 [02:30<00:00,  7.95it/s]


   ✅ 파손 차량 1200장 복사 완료.

[2/3] COCO 정상 차량 다운로드 (FiftyOne)...


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


...

 100% |█████████████████| 700/700 [5.0s elapsed, 0s remaining, 149.2 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 700/700 [5.0s elapsed, 0s remaining, 149.2 samples/s]      


Dataset 'coco-2017-validation-700' created


INFO:fiftyone.zoo.datasets:Dataset 'coco-2017-validation-700' created


   ✅ 배경 600장 수집 완료

🎉 모든 작업 완료! 저장 위치: /content/drive/MyDrive/03. HDMF/(pre_study)2026_ImageDetectionStudy/1_car_classification/Dataset_No_Resizing


In [ ]:
# [Cell 3] 최종 데이터셋 수량 점검 및 검증
# ---------------------------------------------------------
import os

def check_final_dataset_stats(root_dir):
    print(f"📊 [최종 점검] 데이터셋 저장 경로: {root_dir}")
    print("=" * 60)
    print(f"{'Class (Folder)':<20} | {'Images':<10} | {'Labels':<10} | {'Status'}")
    print("-" * 60)

    # 1. 파손 차량 (Damaged)
    dmg_img_dir = os.path.join(root_dir, "damaged", "images")
    dmg_lbl_dir = os.path.join(root_dir, "damaged", "labels")

    n_dmg_img = len(os.listdir(dmg_img_dir)) if os.path.exists(dmg_img_dir) else 0
    n_dmg_lbl = len(os.listdir(dmg_lbl_dir)) if os.path.exists(dmg_lbl_dir) else 0

    # 상태 확인 (이미지와 라벨 개수가 같아야 함)
    status_dmg = "✅ OK" if n_dmg_img == n_dmg_lbl and n_dmg_img > 0 else "⚠️ Check"
    print(f"{'1. Damaged':<20} | {n_dmg_img:<10} | {n_dmg_lbl:<10} | {status_dmg}")

    # 2. 정상 차량 (Normal)
    norm_img_dir = os.path.join(root_dir, "normal", "images")
    n_norm_img = len(os.listdir(norm_img_dir)) if os.path.exists(norm_img_dir) else 0
    print(f"{'2. Normal':<20} | {n_norm_img:<10} | {'-':<10} | {'✅ OK' if n_norm_img > 0 else '⚠️ Empty'}")

    # 3. 배경 이미지 (Background)
    bg_img_dir = os.path.join(root_dir, "background", "images")
    n_bg_img = len(os.listdir(bg_img_dir)) if os.path.exists(bg_img_dir) else 0
    print(f"{'3. Background':<20} | {n_bg_img:<10} | {'-':<10} | {'✅ OK' if n_bg_img > 0 else '⚠️ Empty'}")

    print("-" * 60)
    total_imgs = n_dmg_img + n_norm_img + n_bg_img
    print(f"🏆 총 이미지 수: {total_imgs}장")
    print("=" * 60)

    if n_dmg_img != n_dmg_lbl:
        print("\n🚨 [경고] 파손 차량의 이미지와 라벨 개수가 일치하지 않습니다!")
        print("   데이터 전송 중 누락이 발생했거나, 매칭되지 않는 파일이 있을 수 있습니다.")

# 실행 (위에서 설정한 TARGET_ROOT 변수 사용)
if 'TARGET_ROOT' in locals():
    check_final_dataset_stats(TARGET_ROOT)
else:
    # 만약 변수가 없다면 직접 경로 입력
    # 예: check_final_dataset_stats("/content/drive/MyDrive/...")
    print("❌ TARGET_ROOT 변수를 찾을 수 없습니다. 위 셀을 먼저 실행했는지 확인해주세요.")

📊 [최종 점검] 데이터셋 저장 경로: /content/drive/MyDrive/03. HDMF/(pre_study)2026_ImageDetectionStudy/1_car_classification/Dataset_No_Resizing
Class (Folder)       | Images     | Labels     | Status
------------------------------------------------------------
1. Damaged           | 1200       | 1200       | ✅ OK
2. Normal            | 157        | -          | ✅ OK
3. Background        | 600        | -          | ✅ OK
------------------------------------------------------------
🏆 총 이미지 수: 1957장
